# Data Enrichment: Record Matching

Record matching involves identifying records in two different data tables that refer to the same entity. This is a crucial task for ensuring accurate data integration, eliminating duplicates, and maintaining high data quality.

Large Language Models (LLMs) bring powerful capabilities to this complex problem. By leveraging the extensive world knowledge embedded in these pre-trained models, organizations can achieve record matching with unparalleled accuracy and efficiency.

Accurate record matching is vital in real-world applications such as data integration, improving data quality, customer relationship management (CRM), healthcare, fraud detection/prevention, and compliance/reporting.

This tutorial demonstrates how to use Cleanlab’s Data Enrichment to reliably match records between two different data tables, achieving higher levels of accuracy than existing methods.

In particular, **Cleanlab's Data Enrichment allows you to boost the recall of your record matching, while meeting precision requirements that you’d otherwise have to apply overly stringent blocking and string-similarity-thresholds to meet.**


Here is a short overview of steps (the user workflow) we are going to follow in this tutorial:

1. Use blocking based on exact string match in particular column(s) to produce an initial crude set of candidate-matches.
2. Filter these candidate-matches to a smaller set based on fuzzy string-similarity measures applied to particular column(s) where matched records are expected to have syntactically similar values.
3. Filter the candidate-matches even further based on Cleanlab's Data Enrichment, which assesses the semantic similarity in column(s) expected to be particularly meaningful.

If we solely rely on blocking and fuzzy string similarity, the resulting predicted matches between records have either too many false positive or negative results. Cleanlab's AI system is instead able to semantically understand each record to better predict the true set of matched records.

## Setup

To run Cleanlab Data Enrichment, make sure you have a [Cleanlab account](https://app.cleanlab.ai/). Sign up for one [here](https://cleanlab.ai/signup/) if you haven't yet. If you've already signed up, check your email for a personal login link.

The Python client can be installed using pip:

In [39]:
!pip install --upgrade cleanlab-studio
!pip install recordlinkage==0.16

Once installed, let's load this package and other dependencies.

In [2]:
import recordlinkage
from recordlinkage.datasets import load_febrl4
import pandas as pd
import numpy as np
import random
random.seed(42)

from cleanlab_studio import Studio
from cleanlab_studio.utils.data_enrichment.enrich import enrich_data


pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", None)

We then initalize our Cleanlab client.

In [3]:
# Get API key from here: https://app.cleanlab.ai/account after creating an account.
studio = Studio("YOUR API KEY HERE")

## Load in data

Now let's load in our data, represented as `Table A` and `Table B`. 

These record tables represent two different tables, where each row represents a person, from the Freely Extensible Biomedical Record Linkage (Febrl) package. As mentioned in the [source](https://recordlinkage.readthedocs.io/en/latest/ref-datasets.html) of the data, the combined dataset of `Table A` and `Table B` has 10000 records (5000 originals and 5000 duplicates, with one duplicate per original) but to generate these two tables, the originals have been split from the duplicates, into `Table A` (containing the 5000 original records) and `Table B` (containing the 5000 duplicate records). We will be using these two data sets for testing record linkage procedures in this tutorial.

In [4]:
!wget -nc https://cleanlab-public.s3.amazonaws.com/Datasets/recordlinkage_data/recordlinkage_table_A.csv
!wget -nc https://cleanlab-public.s3.amazonaws.com/Datasets/recordlinkage_data/recordlinkage_table_B.csv

In [5]:
dfA, dfB = pd.read_csv("recordlinkage_table_A.csv", index_col=[0]), pd.read_csv("recordlinkage_table_B.csv", index_col=[0])
print(dfA.shape, dfB.shape)

(5000, 6) (5000, 4)


In [6]:
# Check what first DataFrame columns look like
dfA.head(1)

,given_name,surname,street_number,street_address,postcode,state
rec_id,,,,,,
rec-1070-org,michaela,neumann,8.0,stanley street,4223,nsw


In [7]:
# Check what second DataFrame columns look like
dfB.head(1)

,given_name,surname,suburb,address_form
rec_id,,,,
rec-561-dup-0,elton,NaN,windermere,my current address is 3 light setreet windermere 3212 vic


## Basic Record Linkage using Blocking + Fuzzy String Matching

As mentioned in [the official recordlinkage package tutorial](https://recordlinkage.readthedocs.io/en/latest/guides/link_two_dataframes.html), we can use the `recordlinkage` library to link two datasets.

The steps presented in the tutorial are:

1. **Indexation:** Candidate pairs are generated by exact matching on the given_name column in `Table A` and `Table B`.
2. **Comparison:** Similarity features are computed for candidate pairs, with exact matching on given_name and fuzzy matching on surname.
3. **Classification:** Pairs matching on all criteria are selected as final matches.


For fuzzy string matching, we make use of the [Jaro-Winkler similarity](https://en.wikipedia.org/wiki/Jaro%E2%80%93Winkler_distance).

In [8]:
# Indexation step
blocking_columns = ["given_name"]  # Columns to block for exact match between datasets
filtering_columns = ["surname"]  # Columns to filter by for fuzzy string matching between datasets
filter_threshold = 0.95  # Threshold of similarity for fuzzy string matching

indexer = recordlinkage.Index()
indexer.block(blocking_columns)
candidate_links = indexer.index(dfA, dfB)
print(f"Number of matches after exact match blocking: {len(candidate_links)}")

# Comparison string similarity filter step
compare_cl = recordlinkage.Compare()
for column in blocking_columns:
    compare_cl.exact(column, column, label=column)

for column in filtering_columns:
    if column in blocking_columns:
        continue
    compare_cl.string(
        column, column, method="jarowinkler", threshold=filter_threshold, label=column
    )

features = compare_cl.compute(candidate_links, dfA, dfB)
display(features.sum(axis=1).value_counts().sort_index(ascending=False))

# Classification step
matches = features[features.sum(axis=1) == len(blocking_columns) + len(filtering_columns)]

Number of matches after exact match blocking: 77249


2.0     3078
1.0    74171
Name: count, dtype: int64

There are 3078 candidate links with a total score of 2. This means these links have passed both the exact match on `given_name` and the fuzzy string match on `surname`.

There are 74171 candidate links with a total score of 1. This means these links have passed either the exact match on `given_name` or the fuzzy string match on `surname`, but not both.

Let's now use the matched indices from `Table A` and `Table B` we obtained in the classification step prior as a filter to obtain our possible matches after fuzzy string matching and exact match blocking.

In [9]:
dfA_index = matches.index.get_level_values('rec_id_1')
dfB_index = matches.index.get_level_values('rec_id_2')

dfA_filtered = dfA.loc[dfA_index].reset_index()
dfB_filtered = dfB.loc[dfB_index].reset_index()

df_possible_matches = dfA_filtered.join(dfB_filtered, lsuffix='_a', rsuffix='_b')

In [10]:
print(f"Number of matches after fuzzy string matching and exact match blocking: {len(df_possible_matches)}")

Number of matches after fuzzy string matching and exact match blocking: 3078


## Use Cleanlab's Data Enrichment to filter candidate-matches even further

In [11]:
# Specify prompt for Data Enrichment
prompt = '''
Help me identify if Information1 and Information2 belong to the same person. The information does not need to match exactly. 

If Information1 and Information2 both belong to the same person, return True. Otherwise, return False. 

Please be as accurate as possible, the world depends on it.

Here are some example Information and Output pairs:

Example1:

Information1:
Given Name: mitchell
Surname: maxon
Street Number: 47
Address: 47 edkins street
lochaoair
north ryde, 3355
Suburb: north ryde
Postcode: 3355

Information2:
Given Name: mitchell
Surname: 
Postcode: 3355
State: nsw

Example1 Output: True

Example2:

Information1:
Given Name: sophie
Surname: manson
Street Number: 14
Address: 14 elizabeth xrescen
lochaoair
manorhouse, 3465
Suburb: manorhouse
Postcode: 3465

Information2:
Given Name: sophie
Surname: millar
Postcode: 4592
State: vic

Example2 Output: False

Example3:

Information1:
Given Name: jayed
Surname: millar
Street Number: 6
Address: 6 meredith c ircuit
lochaoair
thornbury, 6014
Suburb: thornbury
Postcode: 6014

Information2:
Given Name: jayed
Surname: millar
Postcode: 
State: vic

Example3 Output: True

....

Now here is Information1 and Information2 to verify:

Information1:
Given Name: {given_name_a}
Surname: {surname_a}
Street Number: {street_number}
Address: {street_address}
Postcode: {postcode}
State: {state}

Information2:
Given Name: {given_name_b}
Surname: {surname_b}
Suburb: {suburb}
Address Form: {address_form}

Output:
'''

### Run Data Enrichment on possible matches so far

To run Cleanlab Data Enrichment, we make use of the `enrich_data` function specified below from our [Python API](https://github.com/cleanlab/cleanlab-studio/blob/main/cleanlab_studio/utils/data_enrichment/enrich.py). We are passing the following arguments:

**Required Arguments:**

- `studio`, our Cleanlab Studio object we instantiated
- `data`, the DataFrame containing the original data we are enriching
- `prompt`, the prompt we want to use to enrich our data

**Optional Arguments:**
- `subset_indices`, what subset of the supplied data rows to generate metadata for. We set this to `None` which means we will run data enrichment on our full dataset for the given input prompt
- `new_column_name`, an optional name for the returned enriched column. Name acts as a prefix appended to all additional columns that are returned
- `**kwargs`, optional keywords arguments, including
  
    - `quality_preset`: we specify `quality_preset="high"` to improve our data enrichment responses at the expense of higher runtimes and higher token cost (the default quality is `medium`, see more details about the various presets [here](https://help.cleanlab.ai/reference/python/studio/\#method-tlm)).
    - `contrain_outputs`: If specified, every entry in the `metadata` column will exactly match one of these values.

In [12]:
%%time

df = enrich_data(
    studio, 
    df_possible_matches, 
    prompt, 
    subset_indices=None, 
    constrain_outputs=["True", "False"],
    quality_preset='high'
)

Querying TLM... 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████|

CPU times: user 17min 26s, sys: 5min 39s, total: 23min 6s
Wall time: 1h 10min 30s


When the Data Enrichment is complete, you will notice three new columns in the dataset: 

- `metadata`, a column with the response that we prompted for (after processing the response with any specified regex operations or constrained output operations)
- `metadata_trustworthiness`, a corresponding trustworthiness score, which quantifies how confident you can be that the response is correct
- `metadata_log`, if `regex` or specific outputs (via `constrain_outputs`) are specified, there will also be a log column which returns the raw response string from the LLM (before any other operations)

Note that the trustworthiness score corresponds to the original response string, and not to response that has been processed with any additional string handling.

We can also join the results with the original DataFrame:

In [13]:
enriched_df = df_possible_matches.join(df)
enriched_df.head()

,rec_id_1,given_name_a,surname_a,street_number,street_address,postcode,state,rec_id_2,given_name_b,surname_b,suburb,address_form,metadata,metadata_trustworthiness,metadata_log
0,rec-2371-org,michaela,dunstone,37.0,deane street,2065,vic,rec-2371-dup-0,michaela,dunstone,woodcroft,37 deane street 2065 vic,True,0.999852,True
1,rec-3024-org,michaela,wilkins,1.0,hoseason street,3782,nsw,rec-3024-dup-0,michaela,wilkns,jamdstown,street 2 \nstate nsw,True,0.650150,True
2,rec-4652-org,michaela,mcandrew,25.0,parker street,3128,vic,rec-4652-dup-0,michaela,mcandrew,avondale hdights,25 parker smreet 3121 vic,True,0.999852,True
3,rec-4795-org,michaela,wasley,10.0,lake place,2192,nsw,rec-4795-dup-0,michaela,wasley,coombabah,"10 lake place 2192 nsw coombabah Once again, I live at: 10 lake place 2192 nsw coombabah",True,0.999852,True
4,rec-1016-org,courtney,painter,12.0,pinkerton circuit,4560,vic,rec-1016-dup-0,courtney,painter,richlands,12 pinkerton circuit 4560 vci richlands,True,0.999852,True


In [14]:
print(enriched_df.shape)

(3078, 15)


Now let's check how many data points we have `True` in the response that we prompted for using Data Enrichment. 

These represent cases where Data Enrichment thinks there are matching records and are the end result for the user.

In [25]:
print(enriched_df.query("metadata == 'True'").shape)

(2895, 15)


### Use Cleanlab Trustworthiness Score to Filter Record Matches

We can also use the [Trustworthiness Score](https://help.cleanlab.ai/tutorials/tlm/#how-does-the-tlm-trustworthiness-score-work) obtained via Data Enrichment to further filter our record matches based on a threshold. 

This threshold can be determined by sorting the results by the trustworthiness score and then looking for a cutoff point in the results when record matches go from true positives (a match) and true negatives (not a match) to false positives and false negatives.


**Note:** We suggest you play around with this threshold to achieve optimal results for your data. For record matches, you should choose a threshold where you see examples that are false positives (matches) around this trustworthiness score threshold.

In [26]:
sorted_enriched_df = enriched_df.sort_values(by="metadata_trustworthiness", ascending=False)
sorted_enriched_df.head()

,rec_id_1,given_name_a,surname_a,street_number,street_address,postcode,state,rec_id_2,given_name_b,surname_b,suburb,address_form,metadata,metadata_trustworthiness,metadata_log
0,rec-2371-org,michaela,dunstone,37.0,deane street,2065,vic,rec-2371-dup-0,michaela,dunstone,woodcroft,37 deane street 2065 vic,True,0.999852,True
1998,rec-1350-org,james,reid,30.0,snodgrass crescent,2261,qld,rec-1350-dup-0,james,reid,belmont,"30 snodgrass crescent 2261 qld belmont Once again, I live at: 30 snodgrass crescent 2261 qld belmont",True,0.999852,True
1898,rec-4937-org,chelsea,blake,21.0,chandler street,2360,nsw,rec-4937-dup-0,chelsea,blake,swan hill,my current address is 21 chandler street swan hill 2306,True,0.999852,True
1899,rec-3491-org,chelsea,malcolmson,12.0,chalker circuit,6233,vic,rec-3491-dup-0,chelsea,malcolmson,moonah west,"12 chalker circuit 6233 vkc moonah west Once again, I live at: 12 chalker circuit 6233 vkc moonah west",True,0.999852,True
1901,rec-967-org,chelsea,miles,23.0,illawarra court,2230,qld,rec-967-dup-0,chelsea,miles,greensborough,23 illawarra court 2204 qld greensborough,True,0.999852,True


Now let's check how many responses have `True` (predicted match) using Data Enrichment and also in which the response trustworthiness score is >= 0.95, which represent the most trustworthy responses from Data Enrichment. 

In [28]:
high_confidence_matches_df = sorted_enriched_df.query("metadata == 'True' and metadata_trustworthiness >= 0.95")
high_confidence_matches_df.head()

,rec_id_1,given_name_a,surname_a,street_number,street_address,postcode,state,rec_id_2,given_name_b,surname_b,suburb,address_form,metadata,metadata_trustworthiness,metadata_log
0,rec-2371-org,michaela,dunstone,37.0,deane street,2065,vic,rec-2371-dup-0,michaela,dunstone,woodcroft,37 deane street 2065 vic,True,0.999852,True
1998,rec-1350-org,james,reid,30.0,snodgrass crescent,2261,qld,rec-1350-dup-0,james,reid,belmont,"30 snodgrass crescent 2261 qld belmont Once again, I live at: 30 snodgrass crescent 2261 qld belmont",True,0.999852,True
1898,rec-4937-org,chelsea,blake,21.0,chandler street,2360,nsw,rec-4937-dup-0,chelsea,blake,swan hill,my current address is 21 chandler street swan hill 2306,True,0.999852,True
1899,rec-3491-org,chelsea,malcolmson,12.0,chalker circuit,6233,vic,rec-3491-dup-0,chelsea,malcolmson,moonah west,"12 chalker circuit 6233 vkc moonah west Once again, I live at: 12 chalker circuit 6233 vkc moonah west",True,0.999852,True
1901,rec-967-org,chelsea,miles,23.0,illawarra court,2230,qld,rec-967-dup-0,chelsea,miles,greensborough,23 illawarra court 2204 qld greensborough,True,0.999852,True


In [29]:
print(high_confidence_matches_df.shape)

(2138, 15)


## Performance Evaluation

As seen above, we already have shown the sets of predicted matches using our approaches of blocking, fuzzy string-matching, and Cleanlab's Data Enrichment on `Table A` and `Table B`. Those predicted matches are the end result of our approach, so the process of calculating potential record matches ends there.

We will now see why this is a good workflow (to achieve high accuracy in record matching) by first showing how to evaluate our performance of record matching. This section is specific to this tutorial and you may not be able to evaluate performance like this in your own applications. Here we happen to know the ground-truth set of matches, so we can evaluate how good our predicted set of matches actually is.

Now we will download and load the set of ground-truth matches (`true_links`) that we will use to evaluate record matching accuracy. Once again, this kind of information will not available for most use cases. The `true_links` are 1:1 matches of a record ID `rec-n-org` for indices ranging from 0 <= n <= 4999 and its corresponding ground truth record `rec-n-dup-0` also for indices ranging from 0 <= n <= 4999 as seen in the following steps.

In [19]:
!wget -nc https://cleanlab-public.s3.amazonaws.com/Datasets/recordlinkage_data/recordlinkage_true_links.csv

For the ground truth matches below, we'll use the multi-index directly which contains the correct pair of records for each match:

In [30]:
# Load in ground-truth set of matches
true_links = pd.read_csv("recordlinkage_true_links.csv").set_index(['Unnamed: 0', 'Unnamed: 1']).index.set_names([None, None])
true_links

MultiIndex([(   'rec-0-org',    'rec-0-dup-0'),
            (   'rec-1-org',    'rec-1-dup-0'),
            (   'rec-2-org',    'rec-2-dup-0'),
            (   'rec-3-org',    'rec-3-dup-0'),
            (   'rec-4-org',    'rec-4-dup-0'),
            (   'rec-5-org',    'rec-5-dup-0'),
            (   'rec-6-org',    'rec-6-dup-0'),
            (   'rec-7-org',    'rec-7-dup-0'),
            (   'rec-8-org',    'rec-8-dup-0'),
            (   'rec-9-org',    'rec-9-dup-0'),
            ...
            ('rec-4990-org', 'rec-4990-dup-0'),
            ('rec-4991-org', 'rec-4991-dup-0'),
            ('rec-4992-org', 'rec-4992-dup-0'),
            ('rec-4993-org', 'rec-4993-dup-0'),
            ('rec-4994-org', 'rec-4994-dup-0'),
            ('rec-4995-org', 'rec-4995-dup-0'),
            ('rec-4996-org', 'rec-4996-dup-0'),
            ('rec-4997-org', 'rec-4997-dup-0'),
            ('rec-4998-org', 'rec-4998-dup-0'),
            ('rec-4999-org', 'rec-4999-dup-0')],
           length=5000)

Now let's calculate the accuracy of our record matching approach from the predicted matches using Cleanlab's Data Enrichment against our ground-truth matches.

In [31]:
# Obtain only record id columns 
de_matches = high_confidence_matches_df.reset_index()[['rec_id_1', 'rec_id_2']]

In [32]:
# Convert predicted matches to MultiIndex
de_predicted_links = pd.MultiIndex.from_frame(de_matches)

In [1]:
# Evaluate using the recordlinkage library's evaluation functions
de_precision = recordlinkage.precision(true_links, de_predicted_links)
de_recall = recordlinkage.recall(true_links, de_predicted_links)
de_fscore = recordlinkage.fscore(true_links, de_predicted_links)

# Print the metrics
print(f"Cleanlab Precision: {de_precision:.2f}")
print(f"Cleanlab Recall: {de_recall:.2f}")
print(f"Cleanlab F1-Score: {de_fscore:.2f}")

Cleanlab Precision: 0.99
Cleanlab Recall: 0.42
Cleanlab F1-Score: 0.59


## Another round of fuzzy string matching

We now have created a set of possible matches with `df_possible_matches` so far using blocking + fuzzy string matching, obtained our set of high confidence matches (`high_confidence_matches_df`) via Cleanlab's Data Enrichment, and evaluated the performance of Cleanlab's Data Enrichment for record matching against ground-truth matches (that you have available in this tutorial but likely do not have available in practice).

Now, to get a similar number of predicted matches as from Cleanlab's Data Enrichment approach (to enable fair comparison), we will run another round of additional fuzzy string matching to find a more curated set of possible matches that can be compared to the possible matches found through Data Enrichment.

We use a `filter_threshold` of 0.99 (on using fuzzy string matching with the `surname` column) to find this curated set that is closer to the number of matches found through Data Enrichment. 

In [34]:
filter_threshold = 0.99  # Threshold of similarity for fuzzy string matching

# Apply fuzzy string matching using recordlinkage.Compare on possible matches
df_A = df_possible_matches[['rec_id_1', 'surname_a']].copy().rename(columns={'rec_id_1': 'rec_id', 'surname_a': 'surname'})
df_B = df_possible_matches[['rec_id_2', 'surname_b']].copy().rename(columns={'rec_id_2': 'rec_id', 'surname_b': 'surname'})

# Create a new index based on the current df_possible_matches
candidate_links_possible = pd.MultiIndex.from_arrays([df_A.index, df_B.index])

compare_cl = recordlinkage.Compare()
compare_cl.string('surname', 'surname', method='jarowinkler', threshold=filter_threshold, label='surname')

# Compute features for possible matches
features_possible = compare_cl.compute(candidate_links_possible, df_A, df_B)

# Filter matches based on the fuzzy string matching
matches_possible = features_possible[features_possible.sum(axis=1) >= 1]

print(f"Number of matches after additional fuzzy string matching: {len(matches_possible)}")

# Convert matches to a DataFrame
matches_df_possible = matches_possible.index.to_frame(index=False)

# Ensure the indices are correctly mapped to the original rec_ids
matches_df_possible['rec_id_1'] = df_possible_matches.loc[matches_df_possible[0], 'rec_id_1'].values
matches_df_possible['rec_id_2'] = df_possible_matches.loc[matches_df_possible[1], 'rec_id_2'].values

Number of matches after additional fuzzy string matching: 2574


In [35]:
predicted_matches = matches_df_possible.reset_index()[['rec_id_1', 'rec_id_2']]
predicted_matches.head()

,rec_id_1,rec_id_2
0,rec-2371-org,rec-2371-dup-0
1,rec-4652-org,rec-4652-dup-0
2,rec-4795-org,rec-4795-dup-0
3,rec-1016-org,rec-1016-dup-0
4,rec-2463-org,rec-2463-dup-0


In [36]:
# Convert predicted matches to MultiIndex
predicted_links = pd.MultiIndex.from_frame(predicted_matches)

### Compare Data Enrichment accuracy to fuzzy string matching accuracy for record matching

In [37]:
subset_predicted_links = predicted_links[:high_confidence_matches_df.shape[0]]

In [2]:
# Evaluate using the recordlinkage library's evaluation functions
precision = recordlinkage.precision(true_links, subset_predicted_links)
recall = recordlinkage.recall(true_links, subset_predicted_links)
fscore = recordlinkage.fscore(true_links, subset_predicted_links)

# Print the metrics
print(f"Additional Fuzzy String Filter Precision: {precision:.2f}")
print(f"Additional Fuzzy String Filter Recall: {recall:.2f}")
print(f"Additional Fuzzy String Filter F1-Score: {fscore:.2f}")

Additional Fuzzy String Filter Precision: 0.89
Additional Fuzzy String Filter Recall: 0.38
Additional Fuzzy String Filter F1-Score: 0.53


We can see that the performance metrics are better across the board when using Cleanlab's Data Enrichment compared to blocking + fuzzy string matching.

To summarize, **Cleanlab's Data Enrichment allows you to boost the recall of your record matching, while meeting precision requirements that you’d otherwise have to apply overly stringent blocking and string-similarity-thresholds to meet.**